In [1]:
# imports
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [2]:
# params
calib_methods = ["RF"] #cal.calib_methods.copy() 
metrics = ["acc", "tce", "brier", "ece"]#cal.metrics.copy()

plot = True
 
params = {
    "runs": 3,
    "n_tree": 100, 
    "depth": [5, 6, 7],
    "data_size": 10000,
    "n_features": 2,
    "oob": False,
    "test_split": 0.3,
    "calib_split": 0.1
}

In [3]:
data_list = []

X, y, tp = dp.make_classification_gaussian_with_true_prob(params["data_size"], params["n_features"], 
                                                          class1_mean_min=0, 
                                                          class1_mean_max=1,

                                                          class2_mean_min=1, 
                                                          class2_mean_max=3, 

                                                          seed=0)

# plot data
# plt.scatter(X[:,0], X[:,1], c=y)
# plt.show()

In [4]:
calib_results_dict = {}
for exp_param in params["depth"]:
    data_dict = {} # results for each data set will be saved in here.
    # Data
    exp_data_name = str(exp_param)
    data_list.append(exp_data_name)

    for seed in range(params["runs"]): # running the same dataset multiple times
        # split the data
        data = cal.split_train_calib_test(exp_data_name, X, y, params["test_split"], params["calib_split"], seed, tp)

        # train models
        models = {}

        # dt = DecisionTreeClassifier().fit(data["x_train"], data["y_train"])
        # models["dt"] = dt

        rf = RandomForestClassifier(n_estimators=params["n_tree"], max_depth=exp_param, oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_pet"] = rf

        irrf_l = IR_RF(n_estimators=params["n_tree"], max_depth=exp_param, oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_l"] = irrf_l

        irrf_ct = IR_RF(n_estimators=params["n_tree"], max_depth=exp_param, oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_ct"] = irrf_ct

        # lr = LogisticRegression(random_state=seed).fit(data["x_train"], data["y_train"])
        # models["LR"] = lr

        # svm = SVC(probability=True, random_state=seed).fit(data["x_train"], data["y_train"])
        # models["SVM"] = svm


        # calibration
        res = cal.model_calibration(models, data, metrics) # res is a dict with all the metrics results as well as RF probs and every calibration method decision for every test data point
        
        data_dict = cal.update_runs(data_dict, res) # calib results for every run for the same dataset is aggregated in data_dict (ex. acc of every run as an array)
        
        if plot:
            cal.plot_probs(exp_data_name, res, data, models, seed, ref_plot_name="RF_pet") 

        calib_results_dict.update(data_dict) # merge results of all datasets together


In [5]:
tables = cal.mean_and_ranking_table(calib_results_dict, metrics, models, data_list, mean_and_rank=True)

In [6]:
tables["acc"]

,RF_pet,RF_l,RF_ct
Data,,,
5,0.789667,0.790111,0.787111
6,0.789222,0.789111,0.789556
7,0.790444,0.790000,0.789333
Mean,0.789778,0.789741,0.788667
Rank,1.666667,2.000000,2.333333


In [7]:
tables["tce"]

,RF_pet,RF_l,RF_ct
Data,,,
5,0.002480,0.002530,0.027311
6,0.002419,0.002378,0.028686
7,0.002987,0.002771,0.027474
Mean,0.002629,0.002560,0.027824
Rank,1.666667,1.333333,3.000000


In [8]:
tables["ece"]

,RF_pet,RF_l,RF_ct
Data,,,
5,0.023637,0.026693,0.131813
6,0.018090,0.019213,0.132729
7,0.019689,0.017748,0.127921
Mean,0.020472,0.021218,0.130821
Rank,1.333333,1.666667,3.000000


In [9]:
tables["brier"]

,RF_pet,RF_l,RF_ct
Data,,,
5,0.146280,0.146348,0.170267
6,0.145964,0.145954,0.171388
7,0.146352,0.146205,0.170018
Mean,0.146199,0.146169,0.170558
Rank,1.666667,1.333333,3.000000
